<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#데이터-로드-및-결측치-제거" data-toc-modified-id="데이터-로드-및-결측치-제거-1">데이터 로드 및 결측치 제거</a></span></li><li><span><a href="#위도,경도-변환할-도로명-주소-리스트-생성" data-toc-modified-id="위도,경도-변환할-도로명-주소-리스트-생성-2">위도,경도 변환할 도로명 주소 리스트 생성</a></span></li><li><span><a href="#카카오-API-활용---위도,-경도-변환" data-toc-modified-id="카카오-API-활용---위도,-경도-변환-3">카카오 API 활용 - 위도, 경도 변환</a></span></li></ul></div>

In [1]:
import requests
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas

import warnings
warnings.filterwarnings(action='ignore')

## 데이터 로드 및 결측치 제거

In [2]:
data = pd.read_csv('data/오피스텔(전월세).csv')
data.head()

,Unnamed: 0,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명
0,15,서울특별시 강남구 개포동,1246-3,1246,3,골든빌오피스텔(1246-3),월세,19.08,201501,5,200,75,2,2003.0,개포로 213
1,15,서울특별시 강남구 개포동,1246-3,1246,3,골든빌오피스텔(1246-3),월세,18.80,201502,2,200,75,2,2003.0,개포로 213
2,15,서울특별시 강남구 개포동,1246-3,1246,3,골든빌오피스텔(1246-3),월세,23.76,201508,20,100,80,4,2003.0,개포로 213
3,15,서울특별시 강남구 개포동,13-3,13,3,대청타워,월세,31.91,201501,3,"1,000",60,18,1997.0,개포로 623
4,15,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,41.47,201504,4,"14,000",0,20,1997.0,개포로 623


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208947 entries, 0 to 208946
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  208947 non-null  int64  
 1   시군구         208947 non-null  object 
 2   번지          208155 non-null  object 
 3   본번          208947 non-null  int64  
 4   부번          208947 non-null  int64  
 5   단지명         208947 non-null  object 
 6   전월세구분       208947 non-null  object 
 7   전용면적(㎡)     208947 non-null  float64
 8   계약년월        208947 non-null  int64  
 9   계약일         208947 non-null  int64  
 10  보증금(만원)     208947 non-null  object 
 11  월세(만원)      208947 non-null  object 
 12  층           208947 non-null  int64  
 13  건축년도        200944 non-null  float64
 14  도로명         208947 non-null  object 
dtypes: float64(2), int64(6), object(7)
memory usage: 23.9+ MB


- 보증금(만원), 월세(만원), 도로명 컬럼에 결측치 없음. 그대로 진행.

## 위도,경도 변환할 도로명 주소 리스트 생성

In [8]:
len(data['도로명'].unique())

3462

In [9]:
add_list = data['도로명'].unique().tolist()
add_list

['개포로 213',
 '개포로 623',
 '논현로18길 10-10',
 '선릉로129길 3-4',
 '도산대로46길 11',
 '학동로 342',
 '학동로 338',
 '논현로 641',
 '언주로 626',
 '선릉로129길 3-6',
 '학동로 224',
 '도산대로 158',
 '봉은사로 307',
 '논현로111길 3',
 '테헤란로92길 13',
 '남부순환로 2927',
 '선릉로90길 56',
 '삼성로85길 42',
 '테헤란로64길 14',
 '선릉로62길 38',
 '도곡로 405',
 '선릉로86길 31',
 '테헤란로64길 8',
 '테헤란로64길 9',
 '테헤란로88길 10',
 '테헤란로 428',
 '강남대로 272',
 '남부순환로 2763',
 '언주로 118',
 '언주로 122',
 '언주로30길 56',
 '언주로30길 57',
 '언주로30길 26',
 '테헤란로83길 11',
 '테헤란로63길 12',
 '봉은사로 448',
 '테헤란로83길 14',
 '선릉로 524',
 '봉은사로 404',
 '봉은사로 507',
 '영동대로 647',
 ' ',
 '봉은사로 477',
 '밤고개로1길 38',
 '광평로56길 8-9',
 '광평로 295',
 '밤고개로1길 10',
 '압구정로2길 6',
 '역삼로2길 7',
 '역삼로2길 9',
 '언주로85길 14',
 '테헤란로4길 34',
 '도곡로3길 19',
 '강남대로 328',
 '강남대로66길 14',
 '강남대로84길 15',
 '테헤란로14길 8',
 '테헤란로52길 15',
 '역삼로7길 17',
 '선릉로89길 16',
 '강남대로84길 33',
 '언주로98길 36',
 '테헤란로4길 15',
 '테헤란로20길 15',
 '테헤란로 113',
 '테헤란로 327',
 '역삼로 204',
 '선릉로87길 14',
 '역삼로4길 16',
 '봉은사로 140',
 '논현로87길 41',
 '테헤란로4길 46',
 '선릉로 431',
 '역삼로 109',
 

## 카카오 API 활용 - 위도, 경도 변환

In [8]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK *****"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['address_name'], document['y'], document['x']]
        except:
            val = [0, 0, 0]
    return val

In [12]:
# 주소 검색 함수 적용
address_to_latlon('개포로 213')

['서울 강남구 개포로 213', '37.4787447572028', '127.046112568695']

In [12]:
# 주소 검색 함수 반복 적용
list1 = []
for i in add_list:
    list1.append(address_to_latlon(i))

In [15]:
data2 = pd.DataFrame(add_list, columns=['도로명'])
data2.head()

,도로명
0,개포로 213
1,개포로 623
2,논현로18길 10-10
3,선릉로129길 3-4
4,도산대로46길 11


In [18]:
data3 = pd.DataFrame(list1, columns=['주소','위도','경도']).drop('주소', axis=1)
data3.head()

,위도,경도
0,37.4787447572028,127.046112568695
1,37.494341198165,127.079647753194
2,37.477252336419,127.046083840064
3,37.5161041468958,127.041071323525
4,37.521291505069,127.036652237866


In [19]:
data4 = pd.concat([data2, data3], axis=1)
data4.head()

,도로명,위도,경도
0,개포로 213,37.4787447572028,127.046112568695
1,개포로 623,37.494341198165,127.079647753194
2,논현로18길 10-10,37.477252336419,127.046083840064
3,선릉로129길 3-4,37.5161041468958,127.041071323525
4,도산대로46길 11,37.521291505069,127.036652237866


In [24]:
data4.to_csv('data/오피스텔_위도경도.csv', index=False)